In [2]:
import requests, pymysql, math, re, time, csv
from bs4 import BeautifulSoup

In [3]:
## 회사 코드 csv불러오기 ##

companycode=list()
with open("companycode.csv",'r') as f:
    reader = csv.reader(f)
    
    for code in reader:
        companycode.append(code[0])
total = len(companycode)

In [4]:
## 회사 정보 크롤링 함수 ##

def crawl(code):
    url = "https://thevc.kr/"
    html = ""
    req = requests.get(url+code)
    if req.status_code == 200 :
        html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    
    brand = []    
    brd_code = code
    brd_link = str()
    service_link = str()
    service_name = str()
    service_info = str()
    service_detail = str()
    
    for info in soup.find_all("div", class_="contact_info info_box"):
        for span in info.find_all("span"):
            href = span.find("a").get("href")
            if href[0:4] == "http":
                brd_link = href
    
    for prdt in soup.find_all("div", class_="info_box product_box"):
        name=prdt.find("span", class_="product_name")
        if name!=None:
            service_name=name.text
        else:
            service_name=None
            
        link = prdt.find("a", class_="product_link")
        if link!=None:
            service_link=link.get("href")
        else:
            service_link=None
            
        detail=prdt.find("p")
        if detail!=None:
            detail=str(detail).replace('<p>','').replace('</p>','').replace('<span>','{').replace('</span>','}').strip()
            service_detail=detail
        else:
            service_detail=None

        for span in prdt.select("div > span"):
            class_=span.get("class")
            if class_==None:
                service_info=span.text
            else:
                service_info=None
        
        brand.append([brd_code, brd_link, service_link, service_name, service_info, service_detail])
    
    return brand

In [5]:
## 회사 정보 배열 ##

comp_info = list()
cnt=0

In [9]:
## 크롤링 함수 실행 ##
startTime = time.time()

for comp in companycode[986:]:
    craw_comp = crawl(comp)
    comp_info+=craw_comp
    cnt+=1
    if cnt%10==0:
        print(total, "中", cnt, '개 완료')

print("--- {0:.2f} seconds ---".format(time.time() - startTime))

2007 中 510 개 완료
2007 中 520 개 완료
2007 中 530 개 완료
2007 中 540 개 완료
2007 中 550 개 완료
2007 中 560 개 완료
2007 中 570 개 완료
2007 中 580 개 완료
2007 中 590 개 완료
2007 中 600 개 완료
2007 中 610 개 완료
2007 中 620 개 완료
2007 中 630 개 완료
2007 中 640 개 완료
2007 中 650 개 완료
2007 中 660 개 완료
2007 中 670 개 완료
2007 中 680 개 완료
2007 中 690 개 완료
2007 中 700 개 완료
2007 中 710 개 완료
2007 中 720 개 완료
2007 中 730 개 완료
2007 中 740 개 완료
2007 中 750 개 완료
2007 中 760 개 완료
2007 中 770 개 완료
2007 中 780 개 완료
2007 中 790 개 완료
2007 中 800 개 완료
2007 中 810 개 완료
2007 中 820 개 완료
2007 中 830 개 완료
2007 中 840 개 완료
2007 中 850 개 완료
2007 中 860 개 완료
2007 中 870 개 완료
2007 中 880 개 완료
2007 中 890 개 완료
2007 中 900 개 완료
2007 中 910 개 완료
2007 中 920 개 완료
2007 中 930 개 완료
2007 中 940 개 완료
2007 中 950 개 완료
2007 中 960 개 완료
2007 中 970 개 완료
2007 中 980 개 완료
2007 中 990 개 완료
2007 中 1000 개 완료
--- 2309.59 seconds ---


In [10]:
print(len(comp_info))
# comp_info

1916


In [12]:
## 중간 저장 (985까지 완료) ##

comp_info2csv = open('company-infomation_0-986.csv', 'w', newline='', encoding='UTF8')

csvwriter = csv.writer(comp_info2csv)
for row in comp_info:
    csvwriter.writerow(row)

comp_info2csv.close()